# ¡Aprende SQL!

Este Notebook te permite practicar SQL. Ejecutaremos las consultas directamente desde Python contra la base de datos Postgres configurada en Docker. Para ello son necesarias una serie de configuraciones previas. Si lo deseas puedes proceder directamente a los ejercicios.

In [47]:
# Importaciones de librerías 
import psycopg2
import pandas as pd

In [48]:
# Configurar la conexión a la base de datos
connection = psycopg2.connect(
    host="localhost",
    port="8090",
    database="sales",
    user="example",
    password="example"
)

In [49]:
# Función para ejecutar consultas SQL
def execute_query(query):
    try:
        cursor = connection.cursor()
        
        # Ejecutar la consulta
        cursor.execute(query)
        
        # Comprobar si la consulta devuelve resultados (SELECT)
        if cursor.description is not None:  # `cursor.description` existe solo en consultas SELECT
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            cursor.close()
            return pd.DataFrame(data, columns=columns)
        
        # Si no es una consulta SELECT, confirmar cambios
        else:
            connection.commit()
            cursor.close()
            return "Consulta ejecutada correctamente."
    
    except Exception as e:
        # En caso de error, deshacer cualquier cambio
        connection.rollback()
        return f"Error: {e}"

### Ejercicios DQL

Antes de realizar los ejercicios vamos a presentar el esquema de la base de datos sobre ventas.

![Mapa conceptual de la BD](../images/database-scheme.png)

Ahora realizarás una serie de ejercicios diseñados para practicar consultas (queries) relacionadas con el contenido de la base de datos, enfocándote en DQL (SELECT).

**Ejercicio 1:** Visualizar el contenido de las tablas salesman, customer y order_item.

In [50]:
# Respuestas a las consultas
query1 = "SELECT * FROM salesman;"
query2 = "SELECT * FROM customer;"
query3 = "SELECT * FROM order_item;"

In [51]:
print(execute_query(query1))
print(execute_query(query2))
print(execute_query(query3))

   salesman_id        name      city commission
0         5001  James Hooq  New York       0.15
1         5002  Nail Knite     Paris       0.13
2         5005    Pit Alex    London       0.11
3         5006     Mc Lyon     Paris       0.14
4         5003  Lauson Hen      None       0.12
5         5007   Paul Adam      Rome       0.13
   customer_id            name        city  grade  salesman_id
0         3002    Nick Rimando    New York  100.0       5001.0
1         3005     Graham Zusi  California  200.0       5002.0
2         3001      Brad Guzan      London    NaN          NaN
3         3004  Fabian Johnson       Paris  300.0       5006.0
4         3007      Brad Davis    San Jose  200.0       5003.0
5         3008    Julian Green      London  300.0       5002.0
6         3009   Geoff Cameron      Berlin  100.0       5002.0
7         3003   Jozy Altidore      Moncow  200.0       5007.0
    order_id  customer_id  salesman_id purch_amt  order_date
0      70001         3005       5002

**Ejercicio 2:** Muestra el número de orden, la fecha de la orden y el monto de compra de las órdenes que serán entregadas por el vendedor con ID 5001.

In [52]:
query = '''
SELECT order_id, order_date, purch_amt FROM order_item 
WHERE salesman_id = 5001;
'''
print(execute_query(query))

   order_id  order_date purch_amt
0     70002  2016-10-05     65.26
1     70005  2016-07-27   2400.60
2     70008  2016-09-10   5760.00


**Ejercicio 3:** Recupera el ID del vendedor de todos los vendedores de la tabla de órdenes sin repeticiones.

In [53]:
query = '''
SELECT DISTINCT order_id from order_item
'''
print(execute_query(query))

    order_id
0      70003
1      70008
2      70005
3      70011
4      70004
5      70010
6      70001
7      70002
8      70007
9      70012
10     70009


**Ejercicio 4:** Muestra todos los clientes que pertenecen a Nueva York o no tienen una calificación superior a 100.

In [54]:
query = '''
SELECT * FROM customer
WHERE city = 'New York' OR grade > 100;
'''
print(execute_query(query))

   customer_id            name        city  grade  salesman_id
0         3002    Nick Rimando    New York    100         5001
1         3005     Graham Zusi  California    200         5002
2         3004  Fabian Johnson       Paris    300         5006
3         3007      Brad Davis    San Jose    200         5003
4         3008    Julian Green      London    300         5002
5         3003   Jozy Altidore      Moncow    200         5007


**Ejercicio 5:** Encuentra a los vendedores cuyos nombres comienzan con la letra "N", tienen "l" como cuarto carácter y el resto puede ser cualquier cosa.

In [55]:
query = ''' 
SELECT * FROM salesman
WHERE name LIKE 'N__l%'
'''

print(execute_query(query))

   salesman_id        name   city commission
0         5002  Nail Knite  Paris       0.13


**Ejercicio 6:** Encuentra al cliente que no tiene ninguna calificación (excepto NULL).

In [56]:
query = ''' 
SELECT * FROM CUSTOMER 
WHERE grade IS NULL;
'''
print(execute_query(query))

   customer_id        name    city grade salesman_id
0         3001  Brad Guzan  London  None        None


**Ejercicio 7:** Encuentra el número de vendedores actualmente registrados para todos sus clientes.

In [57]:
query = ''' 
SELECT name, COUNT(salesman_id) AS num_salesman 
FROM customer
GROUP BY customer_id;
'''
print(execute_query(query))

             name  num_salesman
0    Julian Green             1
1   Jozy Altidore             1
2      Brad Guzan             0
3     Graham Zusi             1
4   Geoff Cameron             1
5      Brad Davis             1
6  Fabian Johnson             1
7    Nick Rimando             1


**Ejercicio 8:** Encuentra el nombre y la ciudad de aquellos clientes y vendedores que viven en la misma ciudad.

In [58]:
# Repasemos las tablas
query = '''
SELECT * FROM customer;
'''
print(execute_query(query))

query = '''
SELECT * FROM salesman;
'''
print(execute_query(query))

   customer_id            name        city  grade  salesman_id
0         3002    Nick Rimando    New York  100.0       5001.0
1         3005     Graham Zusi  California  200.0       5002.0
2         3001      Brad Guzan      London    NaN          NaN
3         3004  Fabian Johnson       Paris  300.0       5006.0
4         3007      Brad Davis    San Jose  200.0       5003.0
5         3008    Julian Green      London  300.0       5002.0
6         3009   Geoff Cameron      Berlin  100.0       5002.0
7         3003   Jozy Altidore      Moncow  200.0       5007.0
   salesman_id        name      city commission
0         5001  James Hooq  New York       0.15
1         5002  Nail Knite     Paris       0.13
2         5005    Pit Alex    London       0.11
3         5006     Mc Lyon     Paris       0.14
4         5003  Lauson Hen      None       0.12
5         5007   Paul Adam      Rome       0.13


In [59]:
# Posibilidad 1: entendiendo que queremos sacar los pares clientes y sus vendedores afincados en la misma ciudad
query = ''' 
SELECT customer.name AS customer_name, customer.city, salesman.name AS salesman_name
FROM customer JOIN salesman ON customer.salesman_id = salesman.salesman_id
WHERE customer.city = salesman.city;
'''
print(execute_query(query))

    customer_name      city salesman_name
0    Nick Rimando  New York    James Hooq
1  Fabian Johnson     Paris       Mc Lyon


In [60]:
# Posibilidad 2: entendiendo que queremos sacar clientes y vendedores de las mismas ciudades (independendimiente de si están asignados o no)
query = '''
SELECT customer.name AS customer_name, salesman.name AS salesman_name, customer.city
FROM customer JOIN salesman ON customer.city = salesman.city;
'''
print(execute_query(query))

    customer_name salesman_name      city
0    Nick Rimando    James Hooq  New York
1      Brad Guzan      Pit Alex    London
2  Fabian Johnson       Mc Lyon     Paris
3  Fabian Johnson    Nail Knite     Paris
4    Julian Green      Pit Alex    London


**Ejercicio 9:** Muestra todas las órdenes realizadas por los clientes que no están ubicados en las mismas ciudades que sus vendedores.

In [61]:
# Primero vamos a hacer un join de las tablas para ver todos los datos y poder comprobar que la consulta que hagamos luego es correcta
query = '''
SELECT order_item.order_id, order_item.order_date, order_item.purch_amt, customer.name AS customer_name, salesman.name AS salesman_name, customer.city AS customer_city, salesman.city AS salesman_city
FROM order_item JOIN customer ON order_item.customer_id = customer.customer_id
JOIN salesman ON customer.salesman_id = salesman.salesman_id
'''
print(execute_query(query))

   order_id  order_date purch_amt   customer_name salesman_name customer_city  \
0     70001  2016-10-05    150.50     Graham Zusi    Nail Knite    California   
1     70002  2016-10-05     65.26    Nick Rimando    James Hooq      New York   
2     70004  2016-08-17    110.50   Geoff Cameron    Nail Knite        Berlin   
3     70007  2016-09-10    948.50     Graham Zusi    Nail Knite    California   
4     70005  2016-07-27   2400.60      Brad Davis    Lauson Hen      San Jose   
5     70008  2016-09-10   5760.00  Fabian Johnson       Mc Lyon         Paris   
6     70010  2016-10-10   1983.43  Fabian Johnson       Mc Lyon         Paris   
7     70003  2016-10-10   2480.40   Geoff Cameron    Nail Knite        Berlin   
8     70012  2016-06-27    250.45    Julian Green    Nail Knite        London   
9     70011  2016-08-17     75.29   Jozy Altidore     Paul Adam        Moncow   

  salesman_city  
0         Paris  
1      New York  
2         Paris  
3         Paris  
4          None  


In [62]:
query = '''
SELECT order_item.order_id, order_item.order_date, order_item.purch_amt, customer.name AS customer_name, salesman.name AS salesman_name, customer.city AS customer_city, salesman.city AS salesman_city
FROM order_item JOIN customer ON order_item.customer_id = customer.customer_id
JOIN salesman ON customer.salesman_id = salesman.salesman_id
WHERE NOT salesman.city = customer.city;
'''
print(execute_query(query))

   order_id  order_date purch_amt  customer_name salesman_name customer_city  \
0     70001  2016-10-05    150.50    Graham Zusi    Nail Knite    California   
1     70004  2016-08-17    110.50  Geoff Cameron    Nail Knite        Berlin   
2     70007  2016-09-10    948.50    Graham Zusi    Nail Knite    California   
3     70003  2016-10-10   2480.40  Geoff Cameron    Nail Knite        Berlin   
4     70012  2016-06-27    250.45   Julian Green    Nail Knite        London   
5     70011  2016-08-17     75.29  Jozy Altidore     Paul Adam        Moncow   

  salesman_city  
0         Paris  
1         Paris  
2         Paris  
3         Paris  
4         Paris  
5          Rome  


**Ejercicio 10:** Muestra todas las órdenes cuyo valor es mayor que el valor promedio de las órdenes del 10 de octubre de 2016.

In [63]:
# Version 1: usando CTE (WITH)
query = ''' 
WITH average_value AS (
    SELECT AVG(purch_amt) AS average_value
    FROM order_item
    WHERE order_date = '2016-10-10'
)
SELECT *
FROM order_item
WHERE purch_amt > (
    SELECT average_value
    FROM average_value
);
'''
print(execute_query(query))

   order_id  customer_id  salesman_id purch_amt  order_date
0     70005         3007         5001   2400.60  2016-07-27
1     70008         3004         5001   5760.00  2016-09-10
2     70003         3009         5003   2480.40  2016-10-10


In [64]:
# Version 2: usando subconsulta
query = '''
SELECT *
FROM order_item
WHERE purch_amt > (
    SELECT AVG(purch_amt)
    FROM order_item
    WHERE order_date = '2016-10-10'
);
'''
print(execute_query(query))

   order_id  customer_id  salesman_id purch_amt  order_date
0     70005         3007         5001   2400.60  2016-07-27
1     70008         3004         5001   5760.00  2016-09-10
2     70003         3009         5003   2480.40  2016-10-10


**Ejercicio 11:** Encuentra el nombre y los IDs de todos los vendedores que tienen más de un cliente.

In [69]:
# Primero veamos cuantos clientes tiene cada vendedor
query = '''
SELECT salesman.salesman_id, COUNT(DISTINCT customer.customer_id) AS num_customers
FROM customer FULL OUTER JOIN salesman ON customer.salesman_id = salesman.salesman_id
GROUP BY salesman.salesman_id;
'''
print(execute_query(query))

   salesman_id  num_customers
0       5001.0              1
1       5002.0              3
2       5003.0              1
3       5005.0              0
4       5006.0              1
5       5007.0              1
6          NaN              1


In [74]:
# Vamos a realizar la consulta
query = '''
SELECT salesman.salesman_id, COUNT(DISTINCT customer.customer_id) AS num_customers
FROM salesman
LEFT JOIN customer ON customer.salesman_id = salesman.salesman_id
GROUP BY salesman.salesman_id
HAVING COUNT(DISTINCT customer.customer_id) > 1;
'''
print(execute_query(query))

   salesman_id  num_customers
0         5002              3


In [77]:
# SI queremos mostrar información específica del vendedor
# Opcion 1
query = '''
SELECT salesman.*, COUNT(DISTINCT customer.customer_id) AS num_customers
FROM salesman LEFT JOIN customer ON customer.salesman_id = salesman.salesman_id
GROUP BY salesman.salesman_id
HAVING COUNT(DISTINCT customer.customer_id) > 1;
'''
print(execute_query(query))

   salesman_id        name   city commission  num_customers
0         5002  Nail Knite  Paris       0.13              3


In [76]:
# Opcion 2
query = '''
WITH customers_per_salesman AS (
    SELECT salesman_id, COUNT(DISTINCT customer_id) AS num_customers
    FROM customer
    GROUP BY salesman_id
)
SELECT 
    salesman.*,
    customers_per_salesman.num_customers
FROM salesman LEFT JOIN customers_per_salesman ON salesman.salesman_id = customers_per_salesman.salesman_id
WHERE customers_per_salesman.num_customers > 1;
'''
print(execute_query(query))

   salesman_id        name   city commission  num_customers
0         5002  Nail Knite  Paris       0.13              3


*Observación: Recuerda los tipos de JOINS* 

![Tipos de JOINS](../images/joins.png)

**Ejercicio 12:** Muestra todas las órdenes que tuvieron montos mayores que alguna de las órdenes de agosto de 2016.

In [84]:
# Antes de realizar la consulta revisamos la tabla de pedidos, ordenando por fecha
query = '''
SELECT * FROM order_item
ORDER BY order_date;
'''
print(execute_query(query))

    order_id  customer_id  salesman_id purch_amt  order_date
0      70012         3008       5002.0    250.45  2016-06-27
1      70005         3007       5001.0   2400.60  2016-07-27
2      70011         3003       5007.0     75.29  2016-08-17
3      70004         3009          NaN    110.50  2016-08-17
4      70009         3001          NaN    270.65  2016-09-10
5      70008         3004       5001.0   5760.00  2016-09-10
6      70007         3005       5002.0    948.50  2016-09-10
7      70001         3005       5002.0    150.50  2016-10-05
8      70002         3002       5001.0     65.26  2016-10-05
9      70010         3004       5006.0   1983.43  2016-10-10
10     70003         3009       5003.0   2480.40  2016-10-10


In [85]:
query = ''' 
WITH purch_amt_date AS (
    SELECT purch_amt
    FROM order_item
    WHERE EXTRACT(YEAR FROM order_date) = 2016 AND EXTRACT(MONTH FROM order_date) = 08
)
SELECT order_item.*
FROM order_item
WHERE order_item.purch_amt > ANY (
    SELECT purch_amt FROM purch_amt_date
);
'''
print(execute_query(query))

   order_id  customer_id  salesman_id purch_amt  order_date
0     70001         3005       5002.0    150.50  2016-10-05
1     70009         3001          NaN    270.65  2016-09-10
2     70004         3009          NaN    110.50  2016-08-17
3     70007         3005       5002.0    948.50  2016-09-10
4     70005         3007       5001.0   2400.60  2016-07-27
5     70008         3004       5001.0   5760.00  2016-09-10
6     70010         3004       5006.0   1983.43  2016-10-10
7     70003         3009       5003.0   2480.40  2016-10-10
8     70012         3008       5002.0    250.45  2016-06-27
